In [50]:
import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import logging

from LoadBatch import *
from SensorClasses import *

debug = False

In [51]:
logging.basicConfig(level=logging.INFO, format='[%(levelname)s] - %(message)s')

### I change every 'nan' that is supposed to be 'Scope X Channel Y' to the correct name (i.e. 'ScopeX ChannelY')

In [52]:
# no clue but it worked
def change_nans(my_header):
    new_columns = my_header
    skip_this = False
    apply_replacement = False

    for idx,name in enumerate(my_header):

        try: 'Scope' in name
        except: pass
        else:
            if 'Scope' in name and 'Channel' in name:
                temp_name = name
                apply_replacement = True
                skip_this = True
            else: apply_replacement = False

        if pd.isna(name) and apply_replacement: skip_this = False
        else: skip_this = True

        if skip_this: continue 

        if apply_replacement: new_columns[idx] = temp_name

    return new_columns

In [53]:
### Read the csv pf the RunLog
data_dir = "/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/Data_TestBeam/2023_May"
df = pd.read_csv(os.path.join(data_dir,"CERN Testbeam May 2023 - RunLog.csv"), skiprows=2, header=None, nrows=354)

### Read the CSV file, but only the first two rows (column names)
df_header = pd.read_csv(os.path.join(data_dir,"CERN Testbeam May 2023 - RunLog.csv"), nrows=2, header=None)
df_header = df_header.transpose()
df_header[0] = change_nans(df_header[0])
### Fill the rest of 'nan'
df_header = df_header.fillna('')

### Create a MultiIndex for the columns using the first two rows
multi_index = pd.MultiIndex.from_frame(df_header, names=[None, None])
df.columns = multi_index

if debug:
    display(df.head())
    display(df.columns)

### Join the two layers of names with underscores and assign them to the dataframe
new_column_names = [f'{level1}_{level2}' for level1, level2 in df.columns]
df.columns = new_column_names

### clear up headers and col_types
df.columns = df.columns.str.replace(' ', '')
new_column_names = [name.lstrip('_') if name.startswith('_') else name for name in df.columns]
df.columns = new_column_names

df['EUDAQ_RunNumber'] = df['EUDAQ_RunNumber'].fillna(0).astype(int)

### remove Bad or Ugly Runs
logging.info(f"""length of df: {len(df)}
{df['GoodRun?_G/B/U'].unique()}""")

# df = df[df["GoodRun?_G/B/U"]!='B']  
df = df[df["GoodRun?_G/B/U"]=='G']

logging.info(f"length of df: {len(df)}")
             
# df.head()

[INFO] - length of df: 354
['G' 'B' nan 'U']
[INFO] - length of df: 295


In [54]:
print(list(df["Batch"].unique()))

[199, 100, 101, 201, 202, 203, 301, 401, 402, 403, 204, 406, 407, 408, 409, 410, 411, 412, 205, 206, 413, 414, 501, 502, 503, 504, 505, 601, 602, 603, 604, 605, 701, 702, 801, 802, 901, 902, 1001, 1002, 1101, 1102, 1201, 1202, 1301, 1302, 1303, 1304, 1305, 1306]


In [55]:
### check info
pd.set_option("display.max_columns", None)

df.head()

,Date,Shifters_NOCOMMASANYWHEREPLEASE:),Time,EUDAQ_RunNumber,Scope1_Timestamp,Scope2_Timestamp,Batch,#Events_Thisrun(triggersfield-noteventbuilder),Total,Av._Rate[Hz],SPSvalues_IntensityT4,Colimators,Countontriggers,Trigger,T:DUT1(left)_T[C]tempA,T:DUT2(right)_T[C]tempB,Humidity:DUT2(right)_Humidity[%]green-right,Diskspaceavailableabove5%?_Y/N,FE-i4_Mask,GoodRun?_G/B/U,Angle_Angle[degree],Scope1Channel1_DUTPos.,Scope1Channel1_Sensor,Scope1Channel1_V[V],Scope1Channel1_I[A],Scope1Channel2_DUTPos.,Scope1Channel2_Sensor,Scope1Channel2_V[V],Scope1Channel2_V_real[V],Scope1Channel2_I[A],Scope1Channel3_DUTPos.,Scope1Channel3_Sensor,Scope1Channel3_V[V],Scope1Channel3_V_real[V],Scope1Channel3_I[A],Scope1Channel4_DUTPos.,Scope1Channel4_Sensor,Scope1Channel4_V[V],Scope1Channel4_V_real[V],Scope1Channel4_I[A],Scope1Channel4_,Scope2Channel1_DUTPos.,Scope2Channel1_Sensor,Scope2Channel1_V[V],Scope2Channel1_I[A],Scope2Channel2_DUTPos.,Scope2Channel2_Sensor,Scope2Channel2_V[V],Scope2Channel2_V_real[V],Scope2Channel2_I[A],Scope2Channel3_DUTPos.,Scope2Channel3_Sensor,Scope2Channel3_V[V],Scope2Channel3_V_real[V],Scope2Channel3_I[A],Scope2Channel4_DUTPos.,Scope2Channel4_Sensor,Scope2Channel4_V[V],Scope2Channel4_V_real[V],Scope2Channel4_I[A],Eventssynced_,Runlogcomments_RunlogcommentsNOCOMMASPLEASE!,Commentaires++_Commentaires++,Scope1oneos,Scope2oneos,Datasavedoneos_Telescopeoneos,Telescopentuples_Telescopentuples
0,May/26/2023,Khuram - Marcello Pozzessere,9:33,9577,1.685177e+09,1.685177e+09,199,267839.0,#REF!,323.0,NaN,NaN,NaN,NaN,-31.0,-31.5,0.7,NaN,10-70;40-300,G,NaN,5.0,MCP,2600,72.7,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1,IMEv3-W12-C3-1-4,-80,-80.0,0.000004,NaN,5.0,MCP,2600,72.7,3,CNM-R15973-W5-D138,-80,-80.0,1.000000e-10,2,CNM-R15973-W4-D168,-80,-80.0,5.000000e-10,1.0,IMEv3-W12-C3-1-5,-80,-80.0,0.000004,NaN,NaN,MCP test runs,NaN,NaN,NaN,NaN
1,May/26/2023,Khuram - Marcello Pozzessere,10:22,9578,1.685179e+09,1.685179e+09,199,327359.0,NaN,347.0,NaN,NaN,NaN,NaN,-31.0,-31.5,0.7,NaN,10-70;40-300,G,NaN,5.0,MCP,2500,69.1,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1,IMEv3-W12-C3-1-4,-80,-80.0,0.000004,NaN,5.0,MCP,2500,69.1,3,CNM-R15973-W5-D138,-80,-80.0,1.000000e-10,2,CNM-R15973-W4-D168,-80,-80.0,5.000000e-10,1.0,IMEv3-W12-C3-1-5,-80,-80.0,0.000004,MCP changed to MCP-1499 (previous MCP-1799),NaN,NaN,NaN,NaN,NaN,NaN
2,May/26/2023,Khuram - Marcello Pozzessere,11:34,9579,1.685183e+09,1.685183e+09,199,302559.0,NaN,340.0,NaN,NaN,NaN,NaN,-31.0,-31.4,0.6,NaN,10-70;40-300,G,NaN,5.0,MCP,2500,69.0,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1,IMEv3-W12-C3-1-4,-80,-80.0,0.000004,NaN,5.0,MCP,2500,69.0,3,CNM-R15973-W5-D138,-80,-80.0,1.000000e-10,2,CNM-R15973-W4-D168,-80,-80.0,5.000000e-10,1.0,IMEv3-W12-C3-1-5,-80,-80.0,0.000004,MCP rotated by 180 degree and no splitte MCP c...,NaN,NaN,NaN,NaN,NaN,NaN
3,May/26/2023,Khuram - Marcello Pozzessere,12:41,9580,1.685184e+09,1.685184e+09,199,178559.0,NaN,269.0,NaN,NaN,NaN,NaN,-31.1,-31.5,1.4,NaN,10-70;40-300,G,NaN,5.0,MCP,2500,69.1,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1,IMEv3-W12-C3-1-4,-80,-80.0,0.000004,NaN,5.0,MCP,2500,69.1,3,CNM-R15973-W5-D138,-80,-80.0,1.000000e-10,2,CNM-R15973-W4-D168,-80,-80.0,5.000000e-10,1.0,IMEv3-W12-C3-1-5,-80,-80.0,0.000004,MCP rotated by 180 degree and no splitte MCP c...,NaN,NaN,NaN,NaN,NaN,NaN
4,May/26/2023,Khuram - Marcello Pozzessere,12:01,9581,1.685184e+09,1.685184e+09,199,312479.0,NaN,337.0,NaN,NaN,NaN,NaN,-31.0,-31.0,1.5,NaN,10-70;40-300,G,NaN,5.0,MCP,2500,69.1,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1,IMEv3-W12-C3-1-4,-80,-80.0,0.000004,NaN,5.0,MCP,2500,69.1,3,CNM-R15973-W5-D138,-80,-80.0,1.000000e-10,2,CNM-R15973-W4-D168,-80,-80.0,5.000000e-10,1.0,IMEv3-W12-C3-1-5,-80,-80.0,0.000004,MCP rotated by 180 degree and no splitte MCP c...,NaN,NaN,NaN,NaN,NaN,NaN


### Old method to extract a dictionary with all sensor names

In [56]:
# now I want to make a nested dictionary that for all Batches, Scopes and Channels gives the sensor model
# Scopes and Channels lists as they appear in the column names
oscilloscopes = ["Scope1", "Scope2"]
channels = ["Channel1", "Channel2", "Channel3", "Channel4"]
dict_sensors = {}

for batch in df["Batch"].unique():
    dict_sensors[batch] = {} # nested dictionary
    idx = np.argwhere(df["Batch"]==batch)[0,0]
    for column_name in df.columns:
        if "Sensor" in column_name:
            for scope in oscilloscopes:
                if scope in column_name:
                    for ch in channels:
                        if ch in column_name: # find the name of the sensor in the appropriate column
                            S = scope.replace("Scope","S") # make the dictionary easier to read
                            CH = ch.replace("Channel", "Ch")
                            try: dict_sensors[batch][S][CH] = df[f"{scope}{ch}_Sensor"].iloc[idx]
                            except: dict_sensors[batch][S] = {}
                            dict_sensors[batch][S][CH] = df[f"{scope}{ch}_Sensor"].iloc[idx]
                            if debug: print(df[f"{scope}{ch}_Sensor"].iloc[idx])
#                             print(dict_sensors[batch][scope][ch])

# Save the sensor list into a file
sensor_list_path = "../testbeam-analysis/sensors_list.pickle"
# with open(sensor_list_path, 'wb') as f:
#     pickle.dump(dict_sensors, f)

# for all unique batches save: dict[batch][ScopeX][ChannelY] = 'sensor_name'

In [57]:
logging.info(f"{dict_sensors.keys()}")
# logging.info(f"{dict_sensors.values()}")

[INFO] - dict_keys([199, 100, 101, 201, 202, 203, 301, 401, 402, 403, 204, 406, 407, 408, 409, 410, 411, 412, 205, 206, 413, 414, 501, 502, 503, 504, 505, 601, 602, 603, 604, 605, 701, 702, 801, 802, 901, 902, 1001, 1002, 1101, 1102, 1201, 1202, 1301, 1302, 1303, 1304, 1305, 1306])


#### I am not removing the excluded runs yet because they are not the same for both Oscilloscopes
Also I need to remove the runs mostly when I 'hadd' all the tree files together

In [71]:
excluded_runs = {}
# runs 9534 and 9535 have been mislabeled (should have been 9634 and 9635, but still missing) 
excluded_runs["S1"] = [9534,9535,9634,9635,
                       9582, 9716, # runs 9582,9716 are missing from S1 and S2 (Ntuples)
                       9828,       # run 9828 is missing from S1 (Ntuples)
                       9910,       # run 9910 is empty in S1 and S2 
                       10071,      # run 10071 is missing from S1 (Ntuples)
                                   # These runs are completely empty (idk why??) 
                       9728, 9729, 9730, 9731, 9732, 9733, 9734, 9735, 9736, 9737, 9738, 9739,
                                    # runs after 10097 are missing from both
                       10110, 10112, 10113, 10114, 10115, 10117,10118,10119,10120,10121,10122,10123,10124]

# runs 9534 and 9535 have been mislabeled (should have been 9634 and 9635, but still missing) 
excluded_runs["S2"] = [9534,9535,9634,9635,
                       9582, 9716, # runs 9582,9716 are missing from S1 and S2 (Ntuples)
                       9785,       # run 9785 is missing from S2 (Ntuples)
                       9910,       # run 9910 is empty in S1 and S2 
                       9936,       # run 9936 is missing from S2 (Ntuples)
                       10057,      # run 10057 is missing from S2 (Ntuple
                                   # These runs are completely empty (idk why??) 
                       9728, 9729, 9730, 9731, 9732, 9733, 9734, 9735, 9736, 9737, 9738, 9739,
                                   # runs after 10097 are missing from both
                       10093, 10093, 10094, 10095, 10096,10110,10112,10113,10114,10115,10117,10118,10119,10120,10121,10122,10123,10124]

# These runs are completely empty (idk why??) 9723, 9724, 9728, 9729, 9730, 9731, 9732, 9733, 9734, 9735, 9736, 9737, 9738


## Improved batches dict

In [59]:
df.head()

,Date,Shifters_NOCOMMASANYWHEREPLEASE:),Time,EUDAQ_RunNumber,Scope1_Timestamp,Scope2_Timestamp,Batch,#Events_Thisrun(triggersfield-noteventbuilder),Total,Av._Rate[Hz],SPSvalues_IntensityT4,Colimators,Countontriggers,Trigger,T:DUT1(left)_T[C]tempA,T:DUT2(right)_T[C]tempB,Humidity:DUT2(right)_Humidity[%]green-right,Diskspaceavailableabove5%?_Y/N,FE-i4_Mask,GoodRun?_G/B/U,Angle_Angle[degree],Scope1Channel1_DUTPos.,Scope1Channel1_Sensor,Scope1Channel1_V[V],Scope1Channel1_I[A],Scope1Channel2_DUTPos.,Scope1Channel2_Sensor,Scope1Channel2_V[V],Scope1Channel2_V_real[V],Scope1Channel2_I[A],Scope1Channel3_DUTPos.,Scope1Channel3_Sensor,Scope1Channel3_V[V],Scope1Channel3_V_real[V],Scope1Channel3_I[A],Scope1Channel4_DUTPos.,Scope1Channel4_Sensor,Scope1Channel4_V[V],Scope1Channel4_V_real[V],Scope1Channel4_I[A],Scope1Channel4_,Scope2Channel1_DUTPos.,Scope2Channel1_Sensor,Scope2Channel1_V[V],Scope2Channel1_I[A],Scope2Channel2_DUTPos.,Scope2Channel2_Sensor,Scope2Channel2_V[V],Scope2Channel2_V_real[V],Scope2Channel2_I[A],Scope2Channel3_DUTPos.,Scope2Channel3_Sensor,Scope2Channel3_V[V],Scope2Channel3_V_real[V],Scope2Channel3_I[A],Scope2Channel4_DUTPos.,Scope2Channel4_Sensor,Scope2Channel4_V[V],Scope2Channel4_V_real[V],Scope2Channel4_I[A],Eventssynced_,Runlogcomments_RunlogcommentsNOCOMMASPLEASE!,Commentaires++_Commentaires++,Scope1oneos,Scope2oneos,Datasavedoneos_Telescopeoneos,Telescopentuples_Telescopentuples
0,May/26/2023,Khuram - Marcello Pozzessere,9:33,9577,1.685177e+09,1.685177e+09,199,267839.0,#REF!,323.0,NaN,NaN,NaN,NaN,-31.0,-31.5,0.7,NaN,10-70;40-300,G,NaN,5.0,MCP,2600,72.7,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1,IMEv3-W12-C3-1-4,-80,-80.0,0.000004,NaN,5.0,MCP,2600,72.7,3,CNM-R15973-W5-D138,-80,-80.0,1.000000e-10,2,CNM-R15973-W4-D168,-80,-80.0,5.000000e-10,1.0,IMEv3-W12-C3-1-5,-80,-80.0,0.000004,NaN,NaN,MCP test runs,NaN,NaN,NaN,NaN
1,May/26/2023,Khuram - Marcello Pozzessere,10:22,9578,1.685179e+09,1.685179e+09,199,327359.0,NaN,347.0,NaN,NaN,NaN,NaN,-31.0,-31.5,0.7,NaN,10-70;40-300,G,NaN,5.0,MCP,2500,69.1,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1,IMEv3-W12-C3-1-4,-80,-80.0,0.000004,NaN,5.0,MCP,2500,69.1,3,CNM-R15973-W5-D138,-80,-80.0,1.000000e-10,2,CNM-R15973-W4-D168,-80,-80.0,5.000000e-10,1.0,IMEv3-W12-C3-1-5,-80,-80.0,0.000004,MCP changed to MCP-1499 (previous MCP-1799),NaN,NaN,NaN,NaN,NaN,NaN
2,May/26/2023,Khuram - Marcello Pozzessere,11:34,9579,1.685183e+09,1.685183e+09,199,302559.0,NaN,340.0,NaN,NaN,NaN,NaN,-31.0,-31.4,0.6,NaN,10-70;40-300,G,NaN,5.0,MCP,2500,69.0,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1,IMEv3-W12-C3-1-4,-80,-80.0,0.000004,NaN,5.0,MCP,2500,69.0,3,CNM-R15973-W5-D138,-80,-80.0,1.000000e-10,2,CNM-R15973-W4-D168,-80,-80.0,5.000000e-10,1.0,IMEv3-W12-C3-1-5,-80,-80.0,0.000004,MCP rotated by 180 degree and no splitte MCP c...,NaN,NaN,NaN,NaN,NaN,NaN
3,May/26/2023,Khuram - Marcello Pozzessere,12:41,9580,1.685184e+09,1.685184e+09,199,178559.0,NaN,269.0,NaN,NaN,NaN,NaN,-31.1,-31.5,1.4,NaN,10-70;40-300,G,NaN,5.0,MCP,2500,69.1,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1,IMEv3-W12-C3-1-4,-80,-80.0,0.000004,NaN,5.0,MCP,2500,69.1,3,CNM-R15973-W5-D138,-80,-80.0,1.000000e-10,2,CNM-R15973-W4-D168,-80,-80.0,5.000000e-10,1.0,IMEv3-W12-C3-1-5,-80,-80.0,0.000004,MCP rotated by 180 degree and no splitte MCP c...,NaN,NaN,NaN,NaN,NaN,NaN
4,May/26/2023,Khuram - Marcello Pozzessere,12:01,9581,1.685184e+09,1.685184e+09,199,312479.0,NaN,337.0,NaN,NaN,NaN,NaN,-31.0,-31.0,1.5,NaN,10-70;40-300,G,NaN,5.0,MCP,2500,69.1,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1.0,IMEv3-W12-C2-2-2,-80,-80.0,0.000004,1,IMEv3-W12-C3-1-4,-80,-80.0,0.000004,NaN,5.0,MCP,2500,69.1,3,CNM-R15973-W5-D138,-80,-80.0,1.000000e-10,2,CNM-R15973-W4-D168,-80,-80.0,5.000000e-10,1.0,IMEv3-W12-C3-1-5,-80,-80.0,0.000004,MCP rotated by 180 degree and no splitte MCP c...,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
### Now using the Batch, Oscilloscope and Sensor classes
oscilloscopes = ["Scope1", "Scope2"]
channels = ["Channel1", "Channel2", "Channel3", "Channel4"]
dict_of_batches = {}

for batch in df["Batch"].unique():    # batch number
    idx = np.argwhere(df["Batch"]==batch)[:,0]  ### take all indices
    if debug:
        print("Batch:", batch)
        print(idx)
    runs_list = []
    for i in idx:     # runs
        ### I still need to figure out how to represent this in a good way
#         runs_list.append(df["EUDAQ_RunNumber"].iloc[i])
        single_run = df["EUDAQ_RunNumber"].iloc[i]
        if (single_run in excluded_runs['S1']) and (single_run in excluded_runs['S2']):   continue
        else:    runs_list.append(df["EUDAQ_RunNumber"].iloc[i])
    if not runs_list: continue ### if all the runs are excluded, so it's empty
    for column_name in df.columns:
        if 'tempA' in column_name:    # tempA
            temperatureA = np.average(df[column_name].iloc[idx])
        if 'tempB' in column_name:    # tempB
            temperatureB = np.average(df[column_name].iloc[idx])
        if 'Angle' in column_name:    # angle
            angle = np.average(df[column_name].iloc[idx].fillna(0))
    S = {}
    for scope in oscilloscopes:   # scopes
        logging.debug(f'Oscilloscope:{scope}')
        this_scope = scope.replace('Scope','S')
#         transimpedance = -1, *get_transimpedance(batch, this_scope) # transimpedance
        S[this_scope] = Oscilloscope(this_scope)    # Oscilloscope object
        for j,ch in enumerate(channels):
            for column_name in df.columns:
                logging.debug(f"{column_name}")
                if (scope in column_name) and (ch in column_name):
                    if 'Sensor' in column_name:            # name
                        name = df[column_name].fillna('').iloc[idx[0]]  ### just take the first one
                    elif 'DUTPos' in column_name:            # dut position
                        dut_position = df[column_name].iloc[idx[0]]
                    elif 'V[V]' in column_name:
                        voltage = df[column_name].iloc[idx[0]]  # voltage
#             this_sensor = Sensor(name, dut_position, transimpedance[j], voltage)
            this_sensor = Sensor(name, dut_position, voltage)
            
            if debug:
                print('name:',name,'board:',board,'dut pos:',dut_position,'fluence:',fluence,'voltage:',voltage)
                print(this_sensor)
            S[this_scope].add_sensor(f"Ch{j+1}",this_sensor)
    dict_of_batches[batch] = Batch(batch,angle,runs_list,temperatureA,temperatureB,S['S1'],S['S2'])
    
    
### Save the sensor list into a file
dict_batches_path = "/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/testbeam-analysis/dict_of_batches.pickle"
with open(dict_batches_path, 'wb') as f:
    pickle.dump(dict_of_batches, f)

logging.info(f"{dict_of_batches.keys()}")


[INFO] - dict_keys([199, 100, 101, 201, 202, 203, 301, 401, 402, 403, 204, 406, 407, 408, 409, 410, 411, 205, 206, 413, 414, 501, 502, 503, 504, 505, 601, 602, 603, 604, 605, 701, 702, 801, 802, 901, 902, 1001, 1002, 1101, 1102, 1201, 1202])


In [83]:
### to which batch the missing runs belong to
for key, value in dict_of_batches.items():
    for single_run in value.runs:
        if (single_run in excluded_runs['S1']) or (single_run in excluded_runs['S2']):
            print(f"run {single_run} missing from batch {key}, only one scope")

run 9785 missing from batch 414, only one scope
run 9828 missing from batch 503, only one scope
run 9936 missing from batch 601, only one scope
run 10057 missing from batch 1002, only one scope
run 10071 missing from batch 1101, only one scope
run 10093 missing from batch 1201, only one scope
run 10093 missing from batch 1201, only one scope
run 10094 missing from batch 1201, only one scope
run 10095 missing from batch 1202, only one scope
run 10096 missing from batch 1202, only one scope


In [89]:
### I need to merge batches 403 and 406 (as notes say)

### dictionary of the runs
I should try to use dict_of_batches instead

In [86]:
# dict_of_runs = {}
# for S in ["S1","S2"]:
#     dict_of_runs[S] = {}
#     for idx, batch in enumerate(df["Batch"]):
#         if df["EUDAQ_RunNumber"].iloc[idx] in excluded_runs[S]: continue
#         if batch not in dict_of_runs[S].keys():
#             dict_of_runs[S][batch] = [df["EUDAQ_RunNumber"].iloc[idx]]
#         else: 
#             dict_of_runs[S][batch].append(df["EUDAQ_RunNumber"].iloc[idx])
            
# # Save the runs dictionary into a file, not really necessary but might as well
# dict_of_runs_path = "/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis//testbeam-analysis/dict_of_runs.pickle"
# with open(dict_of_runs_path, 'w+b') as f:
#     pickle.dump(dict_of_runs, f)

# dict_of_runs

### Using dict_of_runs
(But I should just use the dict_of_batches instead)

In [87]:
# create_combined_tree_files = False ### so I don't accidentally run it everytime

# if create_combined_tree_files:
#     try:
#         os.chdir("/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/Data_TestBeam/2023_May/S1")
#         !pwd
#         for key, value in dict_of_runs["S1"].items():
#             print("batch:", key)
#             output_name = f"tree_May2023_S1_{key}.root"
#             merge_command = f"hadd -f {output_name}"
#             for run in value:
#                 if run<=9999:  merge_command += f" tree00{run}.root"
#                 else:          merge_command += f" tree0{run}.root"
#             print(merge_command)
#             os.system(merge_command)
#     except: print("something wrong")

In [88]:
# if create_combined_tree_files:
#     try:
#         os.chdir('/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/Data_TestBeam/2023_May/S2')
#         !pwd
#         for key, value in dict_of_runs["S2"].items():
#             print("batch:", key)
#             output_name = f"tree_May2023_S2_{key}.root"
#             merge_command = f"hadd -f {output_name}"
#             for run in value:
#                 if run<=9999:  merge_command += f" tree00{run}.root"
#                 else:          merge_command += f" tree0{run}.root"
#             print(merge_command)
#             os.system(merge_command)
#     except: print("something wrong")


## Now using dict_of_batches

In [84]:
create_combined_tree_files = False ### so I don't accidentally run it everytime

try:
    os.chdir("/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/Data_TestBeam/2023_May/S1")
    !pwd
#         for key, value in dict_of_runs["S1"].items():
    for key, value in dict_of_batches.items():
        print("batch:", key)
        output_name = f"tree_May2023_S1_{key}.root"
        merge_command = f"hadd -f {output_name}"
        for run in value.runs:
            if run<=9999:       merge_command += f" tree00{run}.root"
            elif run>=10000:    merge_command += f" tree0{run}.root"
        print(merge_command)
        if create_combined_tree_files:    os.system(merge_command)
except: logging.error("something wrong")

/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/Data_TestBeam/2023_May/S1
batch: 199
hadd -f tree_May2023_S1_199.root tree009577.root tree009578.root tree009579.root tree009580.root tree009581.root
batch: 100
hadd -f tree_May2023_S1_100.root tree009564.root tree009565.root tree009567.root tree009568.root tree009570.root
batch: 101
hadd -f tree_May2023_S1_101.root tree009574.root
batch: 201
hadd -f tree_May2023_S1_201.root tree009587.root tree009588.root tree009590.root
batch: 202
hadd -f tree_May2023_S1_202.root tree009591.root tree009592.root tree009593.root tree009594.root
batch: 203
hadd -f tree_May2023_S1_203.root tree009595.root tree009596.root tree009602.root
batch: 301
hadd -f tree_May2023_S1_301.root tree009609.root tree009611.root tree009613.root tree009614.root tree009615.root tree009616.root
batch: 401
hadd -f tree_May2023_S1_401.root tree009620.root tree009621.root tree009622.root tree009624.root tree009625.root tree009627.root tree009628.root tree009629.root tree0

In [85]:
try:
    os.chdir("/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/Data_TestBeam/2023_May/S2")
    !pwd
#         for key, value in dict_of_runs["S1"].items():
    for key, value in dict_of_batches.items():
        print("batch:", key)
        output_name = f"tree_May2023_S2_{key}.root"
        merge_command = f"hadd -f {output_name}"
        for run in value.runs:
            if run<=9999:       merge_command += f" tree00{run}.root"
            elif run>=10000:    merge_command += f" tree0{run}.root"
        print(merge_command)
        if create_combined_tree_files:    os.system(merge_command)
except: logging.error("something wrong")

/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/Data_TestBeam/2023_May/S2
batch: 199
hadd -f tree_May2023_S2_199.root tree009577.root tree009578.root tree009579.root tree009580.root tree009581.root
batch: 100
hadd -f tree_May2023_S2_100.root tree009564.root tree009565.root tree009567.root tree009568.root tree009570.root
batch: 101
hadd -f tree_May2023_S2_101.root tree009574.root
batch: 201
hadd -f tree_May2023_S2_201.root tree009587.root tree009588.root tree009590.root
batch: 202
hadd -f tree_May2023_S2_202.root tree009591.root tree009592.root tree009593.root tree009594.root
batch: 203
hadd -f tree_May2023_S2_203.root tree009595.root tree009596.root tree009602.root
batch: 301
hadd -f tree_May2023_S2_301.root tree009609.root tree009611.root tree009613.root tree009614.root tree009615.root tree009616.root
batch: 401
hadd -f tree_May2023_S2_401.root tree009620.root tree009621.root tree009622.root tree009624.root tree009625.root tree009627.root tree009628.root tree009629.root tree0

In [90]:
dict_of_batches[401].runs

[9620, 9621, 9622, 9624, 9625, 9627, 9628, 9629, 9630]